# Embedding CRUD Notebook

Este codigo incluye la clase EmbeddingCRUD la cual contiene las funcionalidades CRUD y las importaciones necesarias para el funcionamiento de la misma.

In [1]:
import pandas as pd
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import numpy as np
import random

class EmbeddingCRUD:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.client = QdrantClient(
            url="https://4715095d-53aa-4f96-8cfe-e542e53f8dd8.eu-west-2-0.aws.cloud.qdrant.io:6333", 
            api_key="2Md9iPHFRxTCSdr6a_tgQrk1BJaf2SAfzx6e9qvqaH7aDc-wogWwtw",
            https=True
        )
        self.index_name = "embeddings_index"

    def id_generation():
        return random.radiant(1000, 9999)

    def create(self, text):
        embedding = self.model.encode([text])[0]
        new_id = self.id_generation()
        self.client.upsert(
            collection_name=self.index_name,
            points=[{
                'id': new_id,
                'vector': embedding.tolist(),
                'payload': {'text': text}
            }]
        )
        print(f"Se ha creado un nuevo embedding para el texto: '{new_id, embedding, text}'")

    def read(self):
        embeddings = []
        scroll_response = self.client.scroll(collection_name=self.index_name, limit=1000)

        while scroll_response and scroll_response[0]:
            for point in scroll_response[0]:
                embeddings.append({
                    'id': point.id,
                    'vector': point.vector.tolist(),
                    'payload': point.payload['text']
                })

            scroll_token = scroll_response[1]
            if not scroll_token:
                break
            
            scroll_response = self.client.scroll(collection_name=self.index_name, scroll_token=scroll_token)

        return embeddings

    #def read_similarity(self, query):
    #    query_embedding = self.model.encode(query)
    #    self.df['similarity'] = self.df['embedding'].apply(lambda x: np.dot(x, query_embedding) / (np.linalg.norm(x) * np.linalg.norm(query_embedding)))
    #    print(f"Resultados de la búsqueda para la consulta '{query}':")
    #    return self.df.sort_values('similarity', ascending=False)
    
    def update(self, id, new_text):
        updated_embedding = self.model.encode([new_text])[0]
        self.client.upsert(
            collection_name=self.index_name,
            points=[{
                'id': id,
                'vector': updated_embedding.tolist(),
                'payload': {'text': new_text}
            }]
        )
        print(f"El texto en el id {id} ha sido actualizado a '{new_text}'")

    def delete(self, id):
        self.client.delete(
            collection_name=self.index_name,
            points_selector=[id]
        )
        print(f"Se ha eliminado el índice {id}")

C:\Users\Solvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Example execution file

Instancia de la clase con los metodos CRUD

In [2]:
crud = EmbeddingCRUD()


##### Crear algunos embeddings

In [3]:
crud.create("This is a sample sentence")
crud.create("Another sentence for testing")

Se ha creado un nuevo embedding para el texto: 'This is a sample sentence'
Se ha creado un nuevo embedding para el texto: 'Another sentence for testing'


##### Buscar embeddings similares a una query

In [4]:
results = crud.read("similar sentence")
print(results)

Resultados de la búsqueda para la consulta 'similar sentence':
                           text  \
0     This is a sample sentence   
1  Another sentence for testing   

                                           embedding  similarity  
0  [0.05404346436262131, 0.08422980457544327, 0.0...    0.605037  
1  [-0.006979598663747311, 0.0739775225520134, -0...    0.438689  


##### Actualizar un embedding

In [5]:
crud.update(0, "Updated sentence")

El texto en el índice 0 ha sido actualizado de 'This is a sample sentence' a 'Updated sentence'


##### Eliminar un embedding

In [6]:
crud.delete(1)

Se ha eliminado el texto 'Another sentence for testing' del índice 1
